In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras import layers,losses,optimizers,regularizers
import matplotlib.pyplot as plt 

# Hyper Parameters

In [ ]:
IMAGE_WIDTH=256
IMAGE_HEIGHT=256
PATH="D:\\df\\ai\\facades\\" #dataset Dir
BUFFER_SIZE=400
BACH_SIZE=1
LR=2e-4
B1=0.5
LAMBDA=100

# Build Data Pipeline

In [ ]:
def split_image(image):
    width=tf.shape(image)[1]
    width=int(width/2)
    in_image=image[:,:width, :]
    real_image=image[:,width:,:]
    return in_image,real_image

def resize_image(in_image,real_image):
    in_image=tf.image.resize(in_image,(IMAGE_WIDTH,IMAGE_HEIGHT),tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    real_image=tf.image.resize(real_image,(IMAGE_WIDTH,IMAGE_HEIGHT),tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return in_image,real_image

def normalize_data(in_image,real_image):
    return tf.cast(2*(in_image/255)-1,tf.float32),tf.cast(2*(real_image/255)-1,tf.float32)


def load_data(image_file):
    image = tf.io.read_file(image_file)
    image = tf.io.decode_jpeg(image)
    in_image,real_image=split_image(image)
    in_image,real_image=resize_image(in_image,real_image)
    in_image,real_image=normalize_data(in_image,real_image)
    return real_image,in_image



def normalize_to_normal(x):
    return 255*(x-1)/2



ds_train=tf.data.Dataset.list_files(PATH+"train\\*.jpg",shuffle=False)
ds_train=ds_train.map(load_data,num_parallel_calls=tf.data.AUTOTUNE)
ds_train=ds_train.cache()
ds_train=ds_train.shuffle(buffer_size=BUFFER_SIZE)#400
ds_train=ds_train.batch(batch_size=BACH_SIZE)
ds_train=ds_train.prefetch(tf.data.AUTOTUNE)

ds_test=tf.data.Dataset.list_files(PATH+"val\\*.jpg",shuffle=False)
ds_test=ds_test.map(load_data,num_parallel_calls=tf.data.AUTOTUNE)
ds_test=ds_test.cache()
ds_test=ds_test.batch(batch_size=BACH_SIZE)
ds_test=ds_test.prefetch(tf.data.AUTOTUNE)




# Build Models

## Generator Network


In [ ]:
class Encoder(layers.Layer):
    def __init__(self,channels,use_bachnorm=True,name=None):
        self.use_bachnorm=use_bachnorm
        super(Encoder,self).__init__(name=name)
        self.cn=layers.Conv2D(channels,4,2,padding="same",use_bias=False if use_bachnorm else True)
        self.bn=layers.BatchNormalization()
    def call(self,inputs,training=False):
        x=self.cn(inputs)
        if self.use_bachnorm:
            x=self.bn(x,training=training)
        return tf.nn.leaky_relu(x)


class Decoder(layers.Layer):
    def __init__(self,channels,use_dropout=False,name=None):
        super(Decoder,self).__init__(name=name)
        self.use_dropout=use_dropout
        self.cn=layers.Conv2DTranspose(channels,4,2,padding='same')
        self.bn=layers.BatchNormalization()
        self.drop=layers.Dropout(0.5)
    def call(self,inputs,training=False):
        x=self.cn(inputs)
        x=self.bn(x,training=training)
        if self.use_dropout:
            x=self.drop(x)
        return tf.nn.relu(x)

class Generator(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.down1=Encoder(64,use_bachnorm=False,name="down_1")
        self.down2=Encoder(128,name="down_2")
        self.down3=Encoder(256,name="down_3")
        self.down4=Encoder(512,name="down_4")
        self.down5=Encoder(512,name="down_5")
        self.down6=Encoder(512,name="down_6")
        self.down7=Encoder(512,name="down_7")

        self.bottom=Encoder(512,name="Bottom")
        

        self.up1=Decoder(512,use_dropout=True,name="Up_1")
        self.up2=Decoder(512,use_dropout=True,name="Up_2")
        self.up3=Decoder(512,use_dropout=True,name="Up_3")
        self.up4=Decoder(512,name="Up_4")
        self.up5=Decoder(256,name="Up_5")
        self.up6=Decoder(128,name="Up_6")
        self.up7=Decoder(64,name="Up_7")

        self.last=layers.Conv2DTranspose(3,4,2,"same")

    def call(self,inputs,training=False):
        d1=self.down1(inputs,training=training)
        d2=self.down2(d1,training=training)
        d3=self.down3(d2,training=training)
        d4=self.down4(d3,training=training)
        d5=self.down5(d4,training=training)
        d6=self.down6(d5,training=training)
        d7=self.down7(d6,training=training)

        d8=self.bottom(d7,training=training)
    
        u1=self.up1(d8,training=training)
        u2=self.up2(tf.concat([u1,d7],axis=-1),training=training)
        u3=self.up3(tf.concat([u2,d6],axis=-1),training=training)
        u4=self.up4(tf.concat([u3,d5],axis=-1),training=training)
        u5=self.up5(tf.concat([u4,d4],axis=-1),training=training)
        u6=self.up6(tf.concat([u5,d3],axis=-1),training=training)
        u7=self.up7(tf.concat([u6,d2],axis=-1),training=training)
        u8=self.last(tf.concat([u7,d1],axis=-1),training=training)

        return tf.nn.tanh(u8)
        
    def architecture(self):
        x=keras.Input((256,256,3))
        model=tf.keras.Model(inputs=[x],outputs=self.call(x))
        return model.summary()


generator=Generator()
generator.architecture()
        


## Discriminator Network

In [ ]:

class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.c64=Encoder(64,use_bachnorm=False,name='block_1')
        self.c128=Encoder(128,use_bachnorm=True,name='block_2')
        self.c256=Encoder(256,use_bachnorm=True,name='block_3')
        self.c512=Encoder(512,use_bachnorm=True,name='block_4')
        self.pach=layers.Conv2D(1,(4,4),padding="same")
    def call(self,inputs,targets,training=False):
        data=tf.concat([inputs,targets],axis=-1)
        x=self.c64(data,training=training)
        x=self.c128(x,training=training)
        x=self.c256(x,training=training)
        x=self.c512(x,training=training)
        x=self.pach(x,training=training)
        return tf.nn.sigmoid(x)
    def architecture(self):
        x=keras.Input((256,256,3))
        target=keras.Input((256,256,3))
        model=tf.keras.Model(inputs=[x,target],outputs=self.call(x,target))
        return model.summary()

discriminator=Discriminator()
discriminator.architecture()   


# Train The Models

### Optimizers and Loss Function

* loss function : log(D(x,y))+log(1-D(x,G(x,z)))
* Optimizers : Adam
* learning rate :0.0002
* Beta_1 : 0.5

In [ ]:
loss_fn=losses.BinaryCrossentropy()
gen_opt=optimizers.Adam(LR,B1)
disc_opt=optimizers.Adam(LR,B1)
num_epochs=200

### Lossess

In [ ]:
def generator_loss(gen_image,real_image,disc_gen):
    gen_loss=loss_fn(tf.ones_like(disc_gen),disc_gen)
    l1_loss=tf.reduce_mean(tf.abs(real_image-gen_image))
    loss=gen_loss+LAMBDA*l1_loss
    return loss

def discriminator_loss(disc_gen,disc_real):
    real_loss=loss_fn(tf.ones_like(disc_real),disc_real)
    fake_loss=loss_fn(tf.zeros_like(disc_gen),disc_gen)
    loss=real_loss+fake_loss
    return loss


### Generate images

In [ ]:
def generate_images(model,input, target):
  prediction = model(input, training=True)
  plt.figure(figsize=(15, 15))

  display_list = [input[0], target[0], prediction[0]]
  title = ['Input Image', 'Real Image', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

ds=ds_test.take(10)

for example_input, example_target in ds:
  generate_images(generator, example_input, example_target)

### Training Loop

In [ ]:
import tqdm
from IPython import display

for epoch in range(num_epochs):
    print(f"epoch number: {epoch}...")
    for idx,(inputs,targets) in enumerate(tqdm.tqdm(ds_train)):
        with tf.GradientTape() as gen_tape,tf.GradientTape() as disc_tape:
            pred_img=generator(inputs,training=True)
            disc_pred=discriminator(inputs,pred_img,training=True)
            disc_real=discriminator(inputs,targets,training=True)
            gen_loss=generator_loss(pred_img,targets,disc_pred)
            disc_loss=discriminator_loss(disc_pred,disc_real)
        #Back Propagation
        gen_grads=gen_tape.gradient(gen_loss,generator.trainable_variables)
        gen_opt.apply_gradients(zip(gen_grads,generator.trainable_variables))
        disc_grads=disc_tape.gradient(disc_loss,discriminator.trainable_variables)
        disc_opt.apply_gradients(zip(disc_grads,discriminator.trainable_variables))
    
    
    display.clear_output(wait=True)
    ds=ds_test.take(4)
    for example_input, example_target in ds:
        generate_images(generator, example_input, example_target)
        

In [ ]:

generator.save("D:\\df\\ai\\models\\gan\\fcades\\generator2")
discriminator.save("D:\\df\\ai\\models\\gan\\fcades\\discriminator")